In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
fastballDL = pd.read_csv('DraftLeagueFastball.csv')
fastballMLB = pd.read_csv('MLBFastball.csv')

In [3]:
fastballDL['league'] = 'Draft League'
fastballMLB['league'] = 'MLB'

In [4]:
fastballDL = fastballDL.astype(str)
fastballMLB = fastballMLB.astype(str)

In [5]:
pitchersDL = list(set(fastballDL['pitcher']))

In [6]:
fastball = pd.concat([fastballDL, fastballMLB])

In [7]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [8]:
#fig, ax = plt.subplots(figsize=(100,100))
#sns.heatmap(cosine_sim,cmap='magma',ax=ax)

In [9]:
def get_index_from_pitcher(pitcher):
    return fastball[fastball.pitcher == pitcher].index[0]

In [10]:
def get_pitcher_from_index(index):
    return fastball[fastball.index == index].values[0][0]

In [11]:
for x in pitchersDL:
    
    try:

        fastball = pd.concat([fastballDL, fastballMLB])

        fastball["combined_features"] = fastball.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(fastball["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        fastball = fastball[(fastball['pitcher'] == pitcher_like) | (fastball['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        i=0

        for pitcher in sorted_similar_pitchers:
            print(get_pitcher_from_index(pitcher[0]),", Cosine Sim: ", sorted_similar_pitchers[i][1])
            i=i+1
            if i>1:
                break
                
    except IndexError:
        print(x, ', No Similarity')
        pass

Richardson, Maddex , Cosine Sim:  1.0000000000000002
Richardson, Maddex , No Similarity
Gowens, Riley , Cosine Sim:  0.9999999999999999
Cole, Gerrit , Cosine Sim:  0.4
Cheeley, Austin , Cosine Sim:  1.0000000000000002
Karinchak, James , Cosine Sim:  0.31622776601683794
Mineo, Ricky , Cosine Sim:  1.0
Mineo, Ricky , No Similarity
Johnson, Tyler , Cosine Sim:  0.9999999999999999
Sánchez, Sixto , Cosine Sim:  0.30000000000000004
Garcia, Lex , Cosine Sim:  1.0000000000000002
Buttrey, Ty , Cosine Sim:  0.31622776601683794
Rivera, Geo , Cosine Sim:  1.0
Eickhoff, Jerad , Cosine Sim:  0.4545454545454546
De Grandpre, Cedric , Cosine Sim:  0.9999999999999999
Gilmartin, Sean , Cosine Sim:  0.4472135954999579
Patterson, Kyler , Cosine Sim:  0.9999999999999999
Miller, Andrew , Cosine Sim:  0.3651483716701108
Gonzalez, Jesus , Cosine Sim:  0.9999999999999999
Rasmussen, Drew , Cosine Sim:  0.30000000000000004
Pintaro, Jonathan , Cosine Sim:  1.0000000000000002
McCarthy, Kevin , Cosine Sim:  0.333333

Tonas, Angelo , Cosine Sim:  1.0000000000000002
Conley, Adam , Cosine Sim:  0.3333333333333333
Wendell, Wyatt , Cosine Sim:  1.0000000000000002
Foley, Jason , Cosine Sim:  0.3333333333333333
Jarvis Jr., Evans , Cosine Sim:  0.9999999999999999
Mears, Nick , Cosine Sim:  0.30000000000000004
Reeves, Cobe , Cosine Sim:  1.0000000000000002
Reeves, Cobe , No Similarity
Alba, Max , Cosine Sim:  1.0000000000000002
Alba, Max , No Similarity
Bravo, Zach , Cosine Sim:  0.9999999999999999
Bravo, Zach , No Similarity
Blatter, Brock , Cosine Sim:  1.0000000000000002
Williams, Trevor , Cosine Sim:  0.3333333333333333
Fosbenner, Xavier , Cosine Sim:  1.0000000000000002
Minter, A.J. , Cosine Sim:  0.3333333333333333
Busse, Terry , Cosine Sim:  1.0000000000000002
McRae, Alex , Cosine Sim:  0.3333333333333333
Amerlaan, Ashe , Cosine Sim:  0.9999999999999999
Miley, Wade , Cosine Sim:  0.3651483716701108
Patten, Cole , Cosine Sim:  1.0000000000000002
Gilbert, Tyler , Cosine Sim:  0.408248290463863
Schomber